### Objective

In this notebook, we investigate the Bayesian optimization strategy for PE optimization.

We consider multiple Qs as design specification.

In [16]:
import copy
import time
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from scipy.stats import qmc, norm
import gpflow
from sklearn.metrics import brier_score_loss

import utility
from two_sources import thermal_distribution_maxT
from manager import region_manager

### 0. Setup necessary constants

In [2]:
df_Q = pd.read_csv('./dataset/Q_test_locations.csv')

In [3]:
Tjmax = 175

# Bounds: d, b, L, c, L_duct, n
lb = np.array([5e-3, 73.7e-3, 127.2e-3, 10e-3, 20e-3, 10])
ub = np.array([30e-3, 307e-3, 530e-3, 39e-3, 50e-3, 50])
Data = (25, 50e-3, 65e-3, 61.4e-3, 106e-3)

### 1. Load dataset

In [4]:
df_train = pd.read_csv('./dataset/passive_learning_train.csv')
df_train.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']
print(f"Training pool: {df_train.shape[0]}")

Training pool: 939


In [5]:
df_candidates = pd.read_csv('./dataset/candidates.csv')
df_candidates.columns = ['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
print(f"Candidate pool: {df_candidates.shape[0]}")

Candidate pool: 470070


In [6]:
# Dedicated testing set
df_test = pd.read_csv('./dataset/test.csv')
df_test.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']
print(f"Testing dataset: {df_test.shape[0]}")

Testing dataset: 5655


In [7]:
# Extract train data
X, _, y, _ = utility.create_samples(df_train, 200)
X_test, _, y_test, _ = utility.create_samples(df_test, len(df_test))

# Normalization
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

### 2. BO iterations

#### 2.1 Initial training

In [8]:
%%time

X_train = copy.deepcopy(X)
X_train_scaled = copy.deepcopy(X_scaled)
y_train = copy.deepcopy(y)

GP = utility.fit(X_train_scaled, y_train, n_restarts=10, trainable=True, verbose=True)

Performing 1-th optimization:
Performing 2-th optimization:
Performing 3-th optimization:
Performing 4-th optimization:
Performing 5-th optimization:
Performing 6-th optimization:
Performing 7-th optimization:
Performing 8-th optimization:
Performing 9-th optimization:
Performing 10-th optimization:
CPU times: total: 29.2 s
Wall time: 15 s


In [9]:
X_test_norm = scaler.transform(X_test)
f_mean, f_var = GP.predict_f(X_test_norm, full_cov=False)
y_prob = norm.cdf(175, loc=f_mean, scale=np.sqrt(f_var))
label = np.where(y_test > 175, 1, 0)
brier_score = brier_score_loss(label, 1-y_prob)
    
rmse, max_e, max_per, _, mean_per = utility.evaluate_model(y_test, f_mean.numpy().flatten())
print(f"RMSE: {rmse:.4f} / data std: {np.std(y_test):.4f}")
print(f"Max Error: {max_e:.4f}")
print(f"Max Percentage Error: {max_per:.2f}")
print(f"Mean Percentage Error: {mean_per:.2f}")
print(f"Brier score: {brier_score:.5f}")

RMSE: 12.4605 / data std: 32.8753
Max Error: 168.8406
Max Percentage Error: 58.90
Mean Percentage Error: 7.07
Brier score: 0.02489


#### 2.2 BO strategy specifications

In [10]:
def copy_model(GP):
    model = copy.deepcopy(GP)
    init_lengthscales = model.kernel.lengthscales.numpy().reshape(1, -1)
    init_variance = model.kernel.variance.numpy().flatten()[0]

    return model, init_lengthscales, init_variance

def propose_candidates(model, Q1, Q2, df_candidates, n_batch=10):
    # Compose dataset
    Q1_array = Q1*np.ones((df_candidates.shape[0], 1))
    Q2_array = Q2*np.ones((df_candidates.shape[0], 1))
    X_candidates = df_candidates.to_numpy()
    X_candidates = np.hstack((Q1_array, Q2_array, X_candidates))

    # Evaluate candidates
    f_mean, f_var = utility.GP_predict_candidates(model, X_candidates, scaler)

    # Select candidates
    acq, indices = utility.acquisition(None, f_mean, f_var, X_candidates, scaler, 
                                       Tjmax, batch_mode=True, batch_size=n_batch)
    print("Candidates selected!")

    return X_candidates, acq, indices

In [11]:
def create_managers(X_train, y_train, X_candidates, indices, Data, model,
                   n_batch, Tjmax, Q1, Q2):
    managers = []
    for index in indices:
        # Enrich dataset
        X_train = np.vstack((X_train, X_candidates[index]))
        y, _ = thermal_distribution_maxT(X_candidates[index], Data)
        y_train = np.append(y_train, np.array(y))
    
        # Create managers
        manager = region_manager(model, n_batch, Data, Tjmax, X_candidates[index], Q1, Q2)
        managers.append(manager)
    
        # Update weight
        if y <= manager.Tjmax:
            w = utility.evaluate_weight(X_candidates[index].reshape(1, -1))[0]
        else:
            w = np.inf
            
        manager.init_weight(w) 
    print("Managers created!")

    return managers, X_train, y_train

In [19]:
def branch_and_bound(X_train, y_train, init_lengthscales, init_variance, 
                     managers, indices, lb, ub, scaler):

    n_batch = len(managers)
    
    # Branch and bound
    counter = 0
    status_list = [False] * n_batch
    design_list, weight_list = [], []

    while np.sum(status_list) < n_batch:
        counter += 1
        print(f"Start {counter}-th iteration:")
    
        # Update global GP model
        X_train_scaled = scaler.transform(X_train)
        model = utility.fit(X_train_scaled, y_train, n_restarts=1, init_lengthscales=init_lengthscales.flatten(), 
                            init_variance=init_variance, trainable=False, verbose=False)
        print("Global model updated!")
        
        for i, (manager, index) in enumerate(zip(managers, indices)):
            print(f"Process {i+1}/{n_batch} manager:")
            print("Updating manager model:")
            manager.update_model(model)
            
            if not manager.converge_flag:
                X, y = manager.branch_and_bound(lb, ub, scaler, candidate_num=5000)
                X_train = np.vstack((X_train, X))
                y_train = np.append(y_train, np.array(y))
        
                # 5-Check conditions
                design, weight, converge_flag = manager.check_converge()   
                status_list[i] = converge_flag
    
                if design is not None:
                    design_list.append(design)
                    weight_list.append(weight)
                    print(f"{i+1}/{n_batch} manager done!")
            else:
                print(f"Skip {i+1}/{n_batch} manager.")

    print(f"All managers done!")

    return design_list, weight_list

In [33]:
def parse_results(design_list, weight_list, Data):
    best_weight = np.min(weight_list)
    best_design = design_list[np.argmin(weight_list)]
    Tmax, _ = thermal_distribution_maxT(best_design, Data)

    return best_weight, best_design[2:], Tmax

In [34]:
def BO_run(X_train, y_train, GP, Q1, Q2, df_candidates, n_batch, 
           Data, lb, ub, scaler):
    # Init model
    model, init_lengthscales, init_variance = copy_model(GP)

    # Propose candidates
    X_candidates, acq, indices = propose_candidates(model, Q1, Q2, df_candidates, n_batch)

    # Set up managers (and enrich dataset)
    managers, X_train, y_train = create_managers(X_train, y_train, X_candidates, indices, 
                                                 Data, model, n_batch, Tjmax, Q1, Q2)

    # Branch and bound algorithm
    designs, weights = branch_and_bound(X_train, y_train, init_lengthscales, init_variance, 
                                        managers, indices, lb, ub, scaler)

    # Parse results
    best_weight, best_design, Tmax = parse_results(designs, weights, Data)

    return best_weight, best_design, Tmax

### 3. Multip Qs

In [35]:
n_batch = 10
best_designs = []
best_weights = []
best_Tmaxs = []
runtimes = []
for i, (Q1, Q2) in enumerate(df_Q.to_numpy()[:2]):
    print(f"Processing {i+1}th iteration with Q1 ({Q1:.2f}) & Q2 ({Q2:2f}) combinations:")
    start = time.time()
    best_weight, best_design, Tmax = BO_run(X_train, y_train, GP, Q1, Q2, 
                                            df_candidates, n_batch, Data, lb, ub, 
                                            scaler)
    end = time.time()
    best_designs.append(best_design)
    best_weights.append(best_weight)
    best_Tmaxs.append(Tmax)
    runtimes.append(end-start)

Processing 1th iteration with Q1 (324.55) & Q2 (273.426620) combinations:
Select candidates==>
==> Complete candidates selection.
Candidates selected!
Managers created!
Start 1-th iteration:
Global model updated!
Process 1/10 manager:
Updating manager model:
Generated 1094 new candidates!
Select candidates==>
==> Complete candidates selection.
Process 2/10 manager:
Updating manager model:
Generated 4999 new candidates!
Select candidates==>
==> Complete candidates selection.
Process 3/10 manager:
Updating manager model:
Generated 1531 new candidates!
Select candidates==>
==> Complete candidates selection.
Process 4/10 manager:
Updating manager model:
Generated 3132 new candidates!
Select candidates==>
==> Complete candidates selection.
Process 5/10 manager:
Updating manager model:
Generated 2139 new candidates!
Select candidates==>
==> Complete candidates selection.
5/10 manager done!
Process 6/10 manager:
Updating manager model:
Generated 4994 new candidates!
Select candidates==>
==> C

In [36]:
df = pd.DataFrame({"Q1": df_Q.to_numpy()[:2, 0],
                  "Q2": df_Q.to_numpy()[:2, 1],
                   "weight": best_weights,
                   "Tmax": best_Tmaxs,
                  "Runtime": runtimes})
best_designs = np.array(best_designs)
for i, col in enumerate(['d', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']):
    df[col] = best_designs[:, i]

In [37]:
df

,Q1,Q2,weight,Tmax,Runtime,d,b,L,c,L_duct,n,t,xc1,yc1,xc2,yc2
0,324.548857,273.426620,0.505090,165.899470,46.100153,0.005233,0.077795,0.240320,0.019333,0.041938,11.0,0.001010,0.032526,0.059196,0.038572,0.177734
1,332.350489,70.869323,0.478799,167.778683,41.520903,0.005156,0.076075,0.254258,0.013660,0.031642,11.0,0.001046,0.041270,0.064787,0.031042,0.190260
